In [1]:
import pandas as pd
import os

In [2]:
companies = {
    #Thép
    "Hòa Phát": "HPG",
    "Tập đoàn Hoa Sen": "HSG",
    "CTCP Thép Nam Kim": "NKG",
    "CTCP Tập đoàn Thép Tiến Lên": "TLH",
    "CTCP Tập đoàn Đại Thiên Lộc": "DTL",
    "CTCP Thép VICASA - VNSTEEL": "VCA",
    "CTCP Xi măng Bỉm Sơn": "BCC",
    "CTCP Xi măng Vicem Hoàng Mai": "HOM",
    "CTCP Gạch men Vicenza": "VGC",
    "CTCP VLXD Viglacera Thăng Long": "TLT",
    "CTCP VLXD Viglacera Đông Triều": "DTC",
    "CTCP Tập đoàn Phú Tài": "PTB",
    "CTCP Hóa chất Cơ bản miền Nam": "CSV",
    "CTCP Hóa chất Đức Giang – Lào Cai": "DGL"
}

In [3]:
# Khởi tạo list chứa dữ liệu từng công ty
all_data = []

# Đọc từng file và gắn mã công ty
for full_name, ticker in companies.items():
    file_path = f"pre-processing/{full_name}_data.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Thêm mã công ty
        df["company_id"] = ticker

        # Chuyển cột thời gian nếu có
        if "quarter" in df.columns:
            df["quarter"] = pd.to_datetime(df["quarter"])

        all_data.append(df)
    else:
        print(f"⚠️ File không tồn tại: {file_path}")

# Gộp toàn bộ dữ liệu
df_all = pd.concat(all_data, ignore_index=True)

# Sắp xếp theo thời gian và công ty
df_all = df_all.sort_values(["company_id", "quarter"])

# Tạo biến mục tiêu P(t+1)
df_all["P_t_plus_1"] = df_all.groupby("company_id")["P"].shift(-1)

# Xoá dòng thiếu dữ liệu đầu vào
df_model = df_all.dropna(subset=["P_t_plus_1", "P/E", "EPS", "D/E", "SIZE"])

df_model.set_index(['company_id', 'quarter'], inplace=True)

df_model

C:\Users\hype1\AppData\Local\Temp\ipykernel_27068\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_27068\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_27068\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_27068\3346459644.py:15: UserWarning: Could not infer for

P       EPS       D/E           P/E  \
company_id quarter                                                      
BCC        2019-01-01     18.851333  0.021797  1.449804  8.648532e+02   
           2019-04-01     18.215374  0.053451  1.473650  3.407879e+02   
           2019-07-01     20.647596  0.016621  1.372924  1.242275e+03   
           2019-10-01     25.879976  0.026707  1.262022  9.690427e+02   
           2020-01-01     24.675584  0.018354  1.331348  1.344414e+03   
...                             ...       ...       ...           ...   
VGC        2020-10-01  24438.281250  0.020025  2.053569  1.220383e+06   
           2021-01-01  33198.360656  0.061836  1.944637  5.368811e+05   
           2021-04-01  32322.881356  0.074885  1.546170  4.316343e+05   
           2021-07-01  32592.187500  0.044513  1.483374  7.321951e+05   
           2021-10-01  45815.625000  0.091398  1.631193  5.012739e+05   

                            SIZE    P_t_plus_1  
company_id quarter                              
BCC        2019-01-01  29.171068     18.215374  
           2019-04-01  29.177555     20.647596  
           2019-07-01  29.178127     25.879976  
           2019-10-01  29.143711     24.675584  
           2020-01-01  29.176198     22.846205  
...                          ...           ...  
VGC        2020-10-01  30.699628  33198.360656  
           2021-01-01  30.698178  32322.881356  
           2021-04-01  30.518619  32592.187500  
           2021-07-01  30.542561  45815.625000  
           2021-10-01  30.722746  52688.888889  

[143 rows x 6 columns]

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Giả sử df_model là DataFrame chứa dữ liệu của bạn
# Khởi tạo StandardScaler
scaler = StandardScaler()

# Chuẩn hóa toàn bộ DataFrame
df_model_scaled = df_model.copy()  # Tạo bản sao để không thay đổi dữ liệu gốc
df_model_scaled[:] = scaler.fit_transform(df_model)

# Kiểm tra lại dữ liệu đã chuẩn hóa
df_model_scaled


P       EPS       D/E       P/E      SIZE  \
company_id quarter                                                        
BCC        2019-01-01 -0.687722 -0.285631  0.100314 -0.304686 -0.167697   
           2019-04-01 -0.687753  0.050745  0.125575 -0.305652 -0.164071   
           2019-07-01 -0.687634 -0.340639  0.018874 -0.303991 -0.163751   
           2019-10-01 -0.687378 -0.233458 -0.098605 -0.304495 -0.182988   
           2020-01-01 -0.687437 -0.322219 -0.025167 -0.303803 -0.164829   
...                         ...       ...       ...       ...       ...   
VGC        2020-10-01  0.508504 -0.304462  0.739891  1.941749  0.686701   
           2021-01-01  0.937630  0.139848  0.624498  0.682692  0.685891   
           2021-04-01  0.894744  0.278520  0.202397  0.488820  0.585525   
           2021-07-01  0.907936 -0.044235  0.135876  1.042474  0.598908   
           2021-10-01  1.555708  0.454005  0.292463  0.617101  0.699624   

                       P_t_plus_1  
company_id quarter                 
BCC        2019-01-01   -0.683961  
           2019-04-01   -0.683847  
           2019-07-01   -0.683602  
           2019-10-01   -0.683658  
           2020-01-01   -0.683744  
...                           ...  
VGC        2020-10-01    0.867790  
           2021-01-01    0.826846  
           2021-04-01    0.839441  
           2021-07-01    1.457867  
           2021-10-01    1.779312  

[143 rows x 6 columns]

In [5]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Pooled OLS
pooled_model = PooledOLS(y, X)
pooled_results = pooled_model.fit(cov_type="robust")

# In kết quả ước lượng
pooled_results

Dep. Variable:,P_t_plus_1,R-squared:,0.5041
Estimator:,PooledOLS,R-squared (Between):,0.5032
No. Observations:,143,R-squared (Within):,-0.2474
Date:,"Fri, May 09 2025",R-squared (Overall):,0.5041
Time:,08:56:17,Log-likelihood,-152.76
Cov. Estimator:,Robust,,
,,F-statistic:,35.066
Entities:,12,P-value,0.0000
Avg Obs:,11.917,Distribution:,"F(4,138)"
Min Obs:,4.0000,,
Max Obs:,23.000,F-statistic (robust):,39.468


In [6]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Fixed Effects (FEM)
fixed_effects_model = PanelOLS(y, X, entity_effects=True)  # entity_effects=True để có hiệu ứng cố định theo công ty
fixed_effects_results = fixed_effects_model.fit(cov_type="robust")

# In kết quả ước lượng
fixed_effects_results


Dep. Variable:,P_t_plus_1,R-squared:,0.2902
Estimator:,PanelOLS,R-squared (Between):,-0.5556
No. Observations:,143,R-squared (Within):,0.2902
Date:,"Fri, May 09 2025",R-squared (Overall):,-0.3636
Time:,08:56:17,Log-likelihood,-37.875
Cov. Estimator:,Robust,,
,,F-statistic:,12.983
Entities:,12,P-value,0.0000
Avg Obs:,11.917,Distribution:,"F(4,127)"
Min Obs:,4.0000,,
Max Obs:,23.000,F-statistic (robust):,11.834


In [7]:
from linearmodels.panel import RandomEffects
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Random Effects
random_effects_model = RandomEffects(y, X)
random_effects_results = random_effects_model.fit(cov_type="robust")

# In kết quả ước lượng
random_effects_results


Dep. Variable:,P_t_plus_1,R-squared:,0.2700
Estimator:,RandomEffects,R-squared (Between):,0.2237
No. Observations:,143,R-squared (Within):,0.2740
Date:,"Fri, May 09 2025",R-squared (Overall):,0.2330
Time:,08:56:17,Log-likelihood,-46.055
Cov. Estimator:,Robust,,
,,F-statistic:,12.763
Entities:,12,P-value,0.0000
Avg Obs:,11.917,Distribution:,"F(4,138)"
Min Obs:,4.0000,,
Max Obs:,23.000,F-statistic (robust):,10.665


In [8]:
fixed_effects_results.f_statistic

Model F-statistic (homoskedastic)
H0: All parameters ex. constant are zero
Statistic: 12.9828
P-value: 0.0000
Distributed: F(4,127)
WaldTestStatistic, id: 0x272ac6dc710

In [9]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.stats.stattools import durbin_watson
import scipy.linalg as la

# Lấy các hệ số ước lượng và phương sai-covariance của các mô hình
beta_FE = fixed_effects_results.params
beta_RE = random_effects_results.params
var_FE = fixed_effects_results.cov
var_RE = random_effects_results.cov

# Tính toán kiểm định Hausman
diff = beta_FE - beta_RE
diff_cov = var_FE - var_RE

# Kiểm tra tính khả thi của ma trận khác biệt và tính toán giá trị kiểm định Hausman
hausman_stat = diff.T @ la.inv(diff_cov) @ diff
df = len(beta_FE)  # số bậc tự do (số biến độc lập)

# P-value sử dụng phân phối Chi-squared
from scipy.stats import chi2
p_value = 1 - chi2.cdf(hausman_stat, df)

print(f"Hausman Statistic: {hausman_stat}")
print(f"P-value: {p_value}")

Hausman Statistic: 7.598272116656491
P-value: 0.17980966440078938


In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Thêm hằng số vào các biến độc lập
X = add_constant(df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']])

# Tính VIF cho từng biến độc lập
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# In kết quả
print(vif_data)

  Variable       VIF
0    const  1.000000
1      P/E  1.057067
2      EPS  1.230721
3      D/E  1.148813
4     SIZE  1.198914


In [11]:
from statsmodels.stats.stattools import durbin_watson

# Kiểm tra tự tương quan cho mô hình Pooled OLS
pooled_resids = pooled_results.resids
print(f'Durbin-Watson statistic for Pooled OLS: {durbin_watson(pooled_resids)}')

# Kiểm tra tự tương quan cho mô hình FEM
fixed_effects_resids = fixed_effects_results.resids
print(f'Durbin-Watson statistic for FEM: {durbin_watson(fixed_effects_resids)}')

# Kiểm tra tự tương quan cho mô hình REM
random_effects_resids = random_effects_results.resids
print(f'Durbin-Watson statistic for REM: {durbin_watson(random_effects_resids)}')


Durbin-Watson statistic for Pooled OLS: 0.6459058830090242
Durbin-Watson statistic for FEM: 0.6627522978406624
Durbin-Watson statistic for REM: 0.6683064465450631


In [12]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Format kết quả kiểm định
labels = ['LM Statistic', 'LM P-value', 'F-statistic', 'F-statistic P-value']

# Kiểm tra phương sai của sai số thay đổi cho mô hình Pooled OLS
pooled_bp_test = het_breuschpagan(pooled_resids, X)

print("\n📊 Breusch-Pagan test for Pooled OLS:")
for label, value in zip(labels, pooled_bp_test):
    print(f"{label:<25}: {value:.4f}")

## Kiểm tra phương sai của sai số thay đổi cho mô hình FEM
fixed_effects_bp_test = het_breuschpagan(fixed_effects_resids, X)

print("\n📊 Breusch-Pagan test for FEM:")
for label, value in zip(labels, fixed_effects_bp_test):
    print(f"{label:<25}: {value:.4f}")

# Kiểm tra phương sai của sai số thay đổi cho mô hình REM
random_effects_bp_test = het_breuschpagan(random_effects_resids, X)

print("\n📊 Breusch-Pagan test for REM:")
for label, value in zip(labels, random_effects_bp_test):
    print(f"{label:<25}: {value:.4f}")


📊 Breusch-Pagan test for Pooled OLS:
LM Statistic             : 35.8580
LM P-value               : 0.0000
F-statistic              : 11.5464
F-statistic P-value      : 0.0000

📊 Breusch-Pagan test for FEM:
LM Statistic             : 21.0957
LM P-value               : 0.0003
F-statistic              : 5.9703
F-statistic P-value      : 0.0002

📊 Breusch-Pagan test for REM:
LM Statistic             : 17.7995
LM P-value               : 0.0014
F-statistic              : 4.9048
F-statistic P-value      : 0.0010


In [13]:
from statsmodels.stats.diagnostic import het_white

# Format kết quả kiểm định
labels = ['LM Statistic', 'LM P-value', 'F-statistic', 'F-statistic P-value']

# Kiểm tra phương sai của sai số thay đổi cho mô hình Pooled OLS
pooled_white_test = het_white(pooled_resids, X)

print("\n📊 White test for Pooled OLS:")
for label, value in zip(labels, pooled_white_test):
    print(f"{label:<25}: {value:.4f}")

## Kiểm tra phương sai của sai số thay đổi cho mô hình FEM
fixed_effects_white_test = het_white(fixed_effects_resids, X)

print("\n📊 White test for FEM:")
for label, value in zip(labels, fixed_effects_white_test):
    print(f"{label:<25}: {value:.4f}")

# Kiểm tra phương sai của sai số thay đổi cho mô hình REM
random_effects_white_test = het_white(random_effects_resids, X)

print("\n📊 White test for REM:")
for label, value in zip(labels, random_effects_white_test):
    print(f"{label:<25}: {value:.4f}")



📊 White test for Pooled OLS:
LM Statistic             : 90.9996
LM P-value               : 0.0000
F-statistic              : 15.9998
F-statistic P-value      : 0.0000

📊 White test for FEM:
LM Statistic             : 71.2327
LM P-value               : 0.0000
F-statistic              : 9.0747
F-statistic P-value      : 0.0000

📊 White test for REM:
LM Statistic             : 64.4552
LM P-value               : 0.0000
F-statistic              : 7.5028
F-statistic P-value      : 0.0000
